# Synchronous Generator dq 7th order model vs transient stability classical model

## Run C++ examples

In [ ]:
import villas.dataprocessing.readtools as rt
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import re
import numpy as np
import math
import dpsimpy

#%matplotlib widget

begin_idx=0
end_idx=-1

## Fundamental params simulation

### Parametrization

In [ ]:
# General grid parameters
V_nom_MV = 24e3
V_nom_HV = 230e3
nom_freq = 60
ratio = V_nom_MV / V_nom_HV
nom_omega = nom_freq * 2 * math.pi

# Operation point synchronous generator 
set_point_active_power = 300e6
set_point_voltage = 1.05*V_nom_MV

# Breaker
breaker_open = 1e9
breaker_closed = 0.001

# Line 
line_lenght = 100
line_resistance = 1.27e-4 * 529.0 * line_lenght * ratio**2
line_inductance = 9.05e-4 * 529.0 * line_lenght * ratio**2 / nom_omega
line_capacitance = (1.81e-3 / 529.0 * line_lenght / ratio**2) / nom_omega
line_conductance = 8e-2

# Simulation parameters
sim_name = "EMT_SynGenDQ7odTrapez_SMIB_Fault"
final_time = 1.0
time_step = 10e-6
start_time_fault = 0.2

### Powerflow for Initialization

In [ ]:
sim_name_pf = sim_name + "_PF"
dpsimpy.Logger.set_log_dir("logs/" + sim_name_pf)
time_step_pf = final_time
final_time_pf = final_time + time_step_pf

# Components
n1_pf = dpsimpy.sp.SimNode("n1", dpsimpy.PhaseType.Single)
n2_pf = dpsimpy.sp.SimNode("n2", dpsimpy.PhaseType.Single)

# Synchronous generator ideal model
gen_pf = dpsimpy.sp.ph1.SynchronGenerator("SynGen", dpsimpy.LogLevel.debug)
gen_pf.set_parameters(rated_apparent_power=555e6, rated_voltage=24e3,
                        set_point_active_power=set_point_active_power,
                        set_point_voltage=set_point_voltage,
                        powerflow_bus_type=dpsimpy.PowerflowBusType.PV)
gen_pf.set_base_voltage(V_nom_MV)
gen_pf.modify_power_flow_bus_type(dpsimpy.PowerflowBusType.PV)

# Grid bus as Slack
extnet_pf = dpsimpy.sp.ph1.NetworkInjection("Slack", dpsimpy.LogLevel.debug)
extnet_pf.set_parameters(V_nom_MV)
extnet_pf.set_base_voltage(V_nom_MV)
extnet_pf.modify_power_flow_bus_type(dpsimpy.PowerflowBusType.VD)

# Line
line_pf = dpsimpy.sp.ph1.PiLine("PiLine", dpsimpy.LogLevel.debug)
line_pf.set_parameters(line_resistance, line_inductance, line_capacitance, line_conductance)
line_pf.set_base_voltage(V_nom_MV)

# Topology
gen_pf.connect([n1_pf])
line_pf.connect([n1_pf, n2_pf])
extnet_pf.connect([n2_pf])
system_pf = dpsimpy.SystemTopology(nom_freq, [n1_pf, n2_pf], [gen_pf, line_pf, extnet_pf])

# Logging
logger_pf = dpsimpy.Logger(sim_name_pf)
logger_pf.log_attribute("v1", "v", n1_pf)
logger_pf.log_attribute("v2", "v", n2_pf)
logger_pf.log_attribute("v_line", "v_intf", line_pf)
logger_pf.log_attribute("i_line", "i_intf", line_pf)
logger_pf.log_attribute("v_gen", "v_intf", gen_pf)
logger_pf.log_attribute("ig", "i_intf", gen_pf)

# Simulation
sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.debug)
sim_pf.set_system(system_pf)
sim_pf.set_time_step(time_step_pf)
sim_pf.set_final_time(final_time_pf)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.set_solver_component_behaviour(dpsimpy.SolverBehaviour.Initialization)
sim_pf.do_init_from_nodes_and_terminals(False)
sim_pf.add_logger(logger_pf)
sim_pf.run()

### Dynamic Simulation

In [ ]:
dpsimpy.Logger.set_log_dir("logs/"+sim_name)

# Extract relevant powerflow results
init_terminal_volt = np.abs(n1_pf.single_voltage() * dpsimpy.RMS3PH_TO_PEAK1PH)
init_volt_angle = np.angle(n1_pf.single_voltage())
init_active_power = np.real(gen_pf.get_apparent_power())
init_reactive_power = np.imag(gen_pf.get_apparent_power())
init_mech_power = init_active_power

# Nodes
n1 = dpsimpy.emt.SimNode("n1", dpsimpy.PhaseType.ABC)
n2 = dpsimpy.emt.SimNode("n2", dpsimpy.PhaseType.ABC)

# Components
# Synch
gen = dpsimpy.emt.ph3.SynchronGeneratorDQTrapez("SynGen", dpsimpy.LogLevel.debug)
gen.set_parameters_fundamental_per_unit(
    nom_power=555e6, nom_volt=24e3, nom_freq=60, pole_number=2, nom_field_cur=1300,
    Rs=0.003, Ll=0.15, Lmd=1.6599, Lmq=1.61, Rfd=0.0006, Llfd=0.1648, Rkd=0.0284, Llkd=0.1713, Rkq1=0.0062,
    Llkq1=0.7252, Rkq2=0.0237, Llkq2=0.125, inertia=3.7, init_active_power=init_active_power, init_reactive_power=init_reactive_power,
    init_terminal_volt=init_terminal_volt, init_volt_angle=init_volt_angle,
    init_mech_power=init_mech_power
)

# Grid bus as Slack
extnet = dpsimpy.emt.ph3.NetworkInjection("Slack", dpsimpy.LogLevel.debug)

# Line
line = dpsimpy.emt.ph3.PiLine("PiLine", dpsimpy.LogLevel.debug)
line.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(line_resistance),
                       dpsimpy.Math.single_phase_parameter_to_three_phase(line_inductance),
                       dpsimpy.Math.single_phase_parameter_to_three_phase(line_capacitance),
                       dpsimpy.Math.single_phase_parameter_to_three_phase(line_conductance))

# Breaker
fault = dpsimpy.emt.ph3.Switch("Br_fault", dpsimpy.LogLevel.debug)
fault.set_parameters(dpsimpy.Math.single_phase_parameter_to_three_phase(breaker_open),
                     dpsimpy.Math.single_phase_parameter_to_three_phase(breaker_closed))
fault.open()

# Topology
gen.connect([n1])
line.connect([n1, n2])
extnet.connect([n2])
fault.connect([dpsimpy.emt.SimNode.gnd, n1])
system = dpsimpy.SystemTopology(nom_freq, [n1, n2], [gen, line, fault, extnet])


# Initialization of dynamic topology
system.init_with_powerflow(system_pf, dpsimpy.Domain.EMT)

# Logging
logger = dpsimpy.Logger(sim_name)
logger.log_attribute("v1", "v", n1)
logger.log_attribute("v2", "v", n2)
logger.log_attribute("v_line", "v_intf", line)
logger.log_attribute("i_line", "i_intf", line)
logger.log_attribute("v_gen", "v_intf", gen)
logger.log_attribute("i_gen", "i_intf", gen)
logger.log_attribute("wr_gen", "w_r", gen)
logger.log_attribute("delta_r", "delta_r", gen)
logger.log_attribute("T_e", "T_e", gen)
logger.log_attribute("T_m", "T_m", gen)

# Events
sw1 = dpsimpy.event.SwitchEvent3Ph(start_time_fault, fault, True)

# Simulation
sim = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.debug)
sim.set_system(system)
sim.set_time_step(time_step)
sim.set_final_time(final_time)
sim.set_domain(dpsimpy.Domain.EMT)
sim.add_logger(logger)
sim.add_event(sw1)
sim.run()

## Results 3ph EMT 7th order model with trapezoidal rule

In [ ]:
work_dir = 'logs/EMT_SynGenDQ7odTrapez_SMIB_Fault/'
log_name = 'EMT_SynGenDQ7odTrapez_SMIB_Fault'
print(work_dir + log_name + '.csv')
ts_emt3ph_DQ7odTrapez = rt.read_timeseries_dpsim(work_dir + log_name + '.csv')

## Network node voltages

In [ ]:
plt.figure(figsize=(12,8))
plt.ylabel('Network node voltage (V)', fontsize=18)

for name in ['v1', 'v2']:
    plt.plot(ts_emt3ph_DQ7odTrapez[name + '_0'].time[begin_idx:end_idx], np.sqrt(3/2)*ts_emt3ph_DQ7odTrapez[name + '_0'].values[begin_idx:end_idx], label=name)
    
plt.legend(fontsize=14)
plt.show()

## Generator terminal voltage

In [ ]:
plt.figure(figsize=(12,8))
plt.ylabel('Generator terminal voltage (V)', fontsize=18)

for name in ['v_gen']:
    plt.plot(ts_emt3ph_DQ7odTrapez[name + '_0'].time[begin_idx:end_idx], np.sqrt(3/2)*ts_emt3ph_DQ7odTrapez[name + '_0'].values[begin_idx:end_idx], label=name)
    
plt.legend(fontsize=14)
plt.show()

## Genrerator terminal Current

In [ ]:
plt.figure(figsize=(12,8))
plt.ylabel('Generator terminal current (V)', fontsize=18)

for name in ['i_gen']:
    plt.plot(ts_emt3ph_DQ7odTrapez[name + '_0'].time[begin_idx:end_idx], np.sqrt(3/2)*ts_emt3ph_DQ7odTrapez[name + '_0'].values[begin_idx:end_idx], label=name)

plt.legend()
plt.show()

## Voltage across line

In [ ]:
plt.figure(figsize=(12,8))

for name in ['v_line']:
    plt.plot(ts_emt3ph_DQ7odTrapez[name + '_0'].time[begin_idx:end_idx], np.sqrt(3/2)*ts_emt3ph_DQ7odTrapez[name + '_0'].values[begin_idx:end_idx], label=name)

plt.legend()
plt.show()

## Current through line

In [ ]:
plt.figure(figsize=(12,8))

for name in ['i_line']:
    plt.plot(ts_emt3ph_DQ7odTrapez[name + '_0'].time[begin_idx:end_idx], np.sqrt(3/2)*ts_emt3ph_DQ7odTrapez[name + '_0'].values[begin_idx:end_idx], label=name)

plt.legend()
plt.show()

## Rotor frequency  $f_r$

In [ ]:
plt.figure(figsize=(12,8))
plt.xlabel('time (s)', fontsize=20)
plt.ylabel('Rotor frequency (Hz)', fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

for name in ['wr_gen']:
    plt.plot(ts_emt3ph_DQ7odTrapez[name].time[begin_idx:end_idx], ts_emt3ph_DQ7odTrapez[name].values[begin_idx:end_idx]*60, label=name)

plt.legend(fontsize=18)
plt.show()

## Rotor angular velocity $\omega _r$

In [ ]:
plt.figure(figsize=(12,8))
plt.ylabel('Rotor angular velocity (rad/s)', fontsize=18)


for name in ['wr_gen']:
    plt.plot(ts_emt3ph_DQ7odTrapez[name].time[begin_idx:end_idx], 2*math.pi*60*ts_emt3ph_DQ7odTrapez[name].values[begin_idx:end_idx], label=name +' Full model (9th order)')

plt.legend()
plt.show()

## Rotor angle $\delta _r$

In [ ]:
plt.figure(figsize=(12,8))
for name in ['delta_r']:
    plt.plot(ts_emt3ph_DQ7odTrapez[name].time[begin_idx:end_idx], ts_emt3ph_DQ7odTrapez[name].values[begin_idx:end_idx]*180/3.14, label=name + ' Full model (9th order)')
